<a href="https://www.kaggle.com/mickaelnarboni/ketodapp-cleaning?scriptVersionId=85223960" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# Context

We're going to work on the database of Open Food Facts that you can find here: https://world.openfoodfacts.org/

Open Food Facts is a non-profit project developed by thousands of volunteers from around the world made by everyone, for everyone.

The goal of this notebook is to have a better understanding of the data, clean our dataframe and propose an application based on our observations and exploratory manipulations.

# Notebook preparation

The current notebook has been written on Kaggle due to our low machine performance on Jupyter or Google Collab, explained by the file size which is over 4 Gb. Some of the basic operations such as reading the dataframe were taking a lot of time and was slowing us down in the process. 

We're starting by defining the various librairies that we're going to use in the notebook such as **pandas** to work on the dataframes, **numpy** for linear algebra operations, **matplotlib** for graphics on the missing values and univariable analysis 

Note that we've imported the **warnings librairy** in oder to ignore the warning message about the file size (currently over 4 GB) when reading the file.


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import warnings
warnings.filterwarnings("ignore") # ignore the warnings about file size
import matplotlib.pyplot as plt
from matplotlib import colors
%matplotlib inline
import seaborn as sns

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Observations

We're starting by importing the file and put it into a dataframe using pandas library with the pd.read_csv() operation. 

**Note:**
* We're using **sep** to split the data by tabulation 
* **error_bad_lines** will drop the rows that are invalid (due to csv written mistakes in the file)
* The argument **low_memory** is used for better allocation of the RAM


We're then creating a new dataframe that is going to include a random 10% of the initial dataframe we previously imported that we call *sample10*.
To do so, we've used .sample() operation from pandas library using frac=0.1 argument to get 10% of the initial *raw_data*.

Next, to have an idea of the columns and rows we're treating here, we display the five first rows of the dataframe we just created using .head() operation.  


In [ ]:
# import des données
raw_data = (pd.read_csv('../input/openfoodfacts/en.openfoodfacts.org.products.csv',sep='\t', error_bad_lines=False, low_memory=False))
sample10 = raw_data.sample(frac=0.1) 
# création d'un sample de 10% du dataset pour alléger la mémoire du notebook lors du nettoyage
sample10.head() # 5 premières colonnes et header

In [ ]:
sample10.describe()

In order to have a better idea of the quality of the data we're treating, we calculate the number of columns, rows, cells, missing values and percentage of missing values out of our dataframe.

To do so, we use basics arithmetics operations on our dataframe using pandas library such as:
* len() that is returning the size of the data
* .isna().sum().sum() returns the total missing valeus of the dataframe
* .format() that allows us to return a certain number of digit for the % of missing values

We can already notice that we have an overall **80% of missing values** that we will have to deal with.

In [ ]:
print('number of columns:',len(sample10.columns)) # return the number of columns 
print('number of rows:',len(sample10)) # return the number of rows
print('number of cells:',len(sample10)*len(sample10.columns))# return the number of data 
print('number of missing values:',sample10.isna().sum().sum()) # return the number of total missing values
print('number of missing values in % : {:.2%}'.format((sample10.isna().sum().sum())/(len(sample10)*len(sample10.columns)))) # return the percentage of missing values




Now we want to dive in deeper in the understanding of the missing values. 
Therefore, we're creating a loop that is going to display for each column, the label, the dtype of the variables and the number of missing values.

To do so, we've used:
* .columns() operation that returns the labels of the columns
* .dtypes argument that is returning the type of the variables 
* isna() operation is returning a boolean of the missing value (either True or False) that we combine with .sum() operation to have to total number of each missing value (NaN) for each columns

In [ ]:
for i in sample10.columns:
    print(i,'||',sample10[i].dtypes,'||', sample10[i].isna().sum(),'||','{:.2%}'.format((sample10.isna().sum()/len(sample10))[i])) 


Now, we want to insert the values above in a new dataframe so it will be easier for us to manipulate the data and use it for data visualization. 

In order to do so, we're using:
* .DataFrame() to create an empty dataframe in which we're going to add the columns we're interested in such as Product name, Data type, Missing values count, % Missing values



In [ ]:
missing_data = pd.DataFrame()
missing_data['Column variable'] = sample10.columns
missing_data['Data type'] = list(sample10.dtypes)
missing_data['Missing values count'] = list(sample10.isna().sum())
missing_data['% Missing values'] = list((sample10.isna().sum()/len(sample10))) 
print(missing_data)

# Data Visualization for Missing Values

We're using the **matplotlib** library to display our graph of missing values.

In order to do so, we:
* Sort our Missing values using .sort_values() operation on our *missing_data* dataframe
* Display a figure using .figure()
* Give a comprehensive label to our axis and title

In the following graph, the **x axis** represents our Column variable axis and the **y axis** represents the % Missing values axis.
We've used various operations found in **matplotlib** library to design our graph.


In [ ]:
# data visualization for missing values of our dataframe called missing_data

to_plot = missing_data.sort_values(by=['% Missing values'])
fig, ax = plt.subplots(figsize=(18, 9))
ax.set_facecolor("#2E2E2E")
plt.title('Representation of the % Missing values per column', fontsize=20, y=1.03)
plt.xlabel('Columns Name', fontsize=14)
plt.ylabel('% Missing values', fontsize=14)
plt.grid(True, color="#93a1a1", alpha=0.05)
plt.xticks(rotation=90) # this line allow us to rotate the x_labels so they can be readable
plt.plot(to_plot['Column variable'],to_plot['% Missing values'],color="#E8FF41"); 


# Decisions for Treating with Missing Values

It seems obvious that we don't need to work with columns that have 100% missing values.
It seems also smart to drop the columns that have missing values above a certain threshold. 
We define this threshold based on the variables we want to keep for our application later, which means: fat_per_100g, proteins_per_100g, carbohydrates_per_100g, countries, product_name.
We notice that the variables don't exceed 21% missing values but we're working on a sample so this number can have small fluctuations so we'll define 25% in case of variations in future samples.

To do so, we use:
* .dropna() in pandas library using the arguments **axis** to work on the columns and **thresh** to drop the columns that have more than 25% of missing values



In [ ]:
data = sample10.dropna(axis=1, thresh= len(sample10)*0.75) 

# axis = 1 will drop columns with missing values, if 0, then it drops the rows by default

for i in data.columns:
    print(i,'||',data[i].dtypes,'||', data[i].isna().sum(),'||','{:.2%}'.format((sample10.isna().sum()/len(sample10))[i])) 
    
# Let's have a look at our new dataframe below


Now, we're iterating the methodology we used previously for displaying our missing values.
We create a new dataframe called *missing_data2* that is containing the columns with 40% or less missing values in their columns, then we plot the result to get a new graph of the dataframe. 

In [ ]:
missing_data2 = pd.DataFrame()
missing_data2['Column variable'] = data.columns
missing_data2['Data type'] = list(data.dtypes)
missing_data2['Missing values count'] = list(data.isna().sum())
missing_data2['% Missing values'] = list(data.isna().sum()/len(data)) 
print(missing_data2)

In [ ]:
# data visualization for missing values of our dataframe called missing_data

to_plot = missing_data2.sort_values(by=['% Missing values'])
fig, ax = plt.subplots(figsize=(18, 9))
ax.set_facecolor("#2E2E2E")
plt.title('Representation of the % Missing values per column after cleaning', fontsize=20, y=1.03)
plt.xlabel('Column names', fontsize=14)
plt.ylabel('% Missing values', fontsize=14)
plt.grid(True, color="#93a1a1", alpha=0.05)
plt.xticks(rotation=90) # this line allow us to rotate the x_labels so they can be readable
plt.plot(to_plot['Column variable'],to_plot['% Missing values'],color="#E8FF41"); 

# Check for entire rows with NaN values

We notice that we don't have any rows in our dataframe that have missing values for each column.

In [ ]:
miss_rows=data[data.isnull().all(axis=1)]

print('Number of rows with entire NaN values is:', len(miss_rows))



# Treating Missing 'product_name' Values 

In our *missing_data2*, we notice that an important number of values is missing in the column named **product_name** and it seems complicated to define our application without the name of the product we're working with. 
We decide to drop the rows that have a missing value on the **product_name** column.  

In [ ]:
drop_data = data.dropna(axis=0, how='any', subset=['product_name'], inplace=False)

missing_data3 = pd.DataFrame()
missing_data3['Column variable'] = drop_data.columns
missing_data3['Data type'] = list(drop_data.dtypes)
missing_data3['Missing values count'] = list(drop_data.isna().sum())
missing_data3['% Missing values'] = list(drop_data.isna().sum()/(len(drop_data))) 
print(missing_data3)

# Duplicated 'product_name' Values Observations

For us, this step is an observation to know how many duplicated values we can find in the column **product_name** so we know we can keep this information for later in case we need it. 

To do so, we're going to create two dataframes:
* Our dataframe **check** is going to return boolean values for each *product_name* to locate if the value got a duplicate in the same column. **True** if a duplicate value exist and **False** if not.
* The next step is about determining the number of duplicate values across the *product_name* column for each cell. In two steps, we firstly create a dataframe called **dups** that contain all the duplicated value from *product_name* in our **check** dataframe. Secondly, we create another dataframe named **count_dups** that we sort by **product_name** and by descending order, that is going to contain the count of duplicated value for each **product_name**.

In [ ]:
check = pd.DataFrame(drop_data['product_name'].duplicated()) # return a dataframe of boolean containing the column product_name and a second column True or False
check = check.set_index(drop_data['product_name']).set_axis(['Duplicated count'], axis='columns') # change the index to the product name to know what products we are talking about
# returns True which means that there are product_name that are duplicated
check.head(10)

In [ ]:
dups = check[check['Duplicated count'] == True]
count_dups = dups.groupby(['product_name'], sort=True).count().sort_values(by=['Duplicated count'], axis=0, ascending=False)
print(count_dups)

# Entire NaN Rows for Quantitative Variables

A step further in our row cleaning, we want to delete the rows that contain only NaN values on the quantitative variables because we won't make much of an application with no quantitative values. 
To do so, we proceed with two dataframes again:
* First, we create a dataframe called **mask** and using the operations *.loc* to locate the columns that interest us, *.isnull()* to target only the missing values on those columns and *.sum()* with the condition that NaN values appear on respectively all our columns, we gather all the rows with missing values on the quantitative variables into it. 
* Then, we create our last clean dataframe called **drop_data_nan** that is a combination of **drop_data** - our original dataframe with the clean columns - and **mask** using the *.append()* operation. Using *.drop_duplicates()* and keeping the option *keep=False* we delete all the duplicates values, which allows us to delete all the rows with NaN that we previously gathered in our **mask** dataframe because they obviously are contained in **drop_data** as well.  

In [ ]:
mask = drop_data[drop_data.loc[:,['energy-kcal_100g','energy_100g','fat_100g','saturated-fat_100g','carbohydrates_100g','sugars_100g','proteins_100g']].isnull().sum(axis=1) == 9]
# mask contains all the rows with NaN values on the quantitative variables

In [ ]:
drop_data_nan = drop_data.append(mask).drop_duplicates(keep=False)
drop_data_nan.info()

In [ ]:
import os
os.chdir(r'./')
drop_data_nan.to_csv('clean_p2.csv',sep = '\t',index = True)

In [ ]:
from IPython.display import FileLink
FileLink(r'clean_p2.csv')